In [1]:
### Libraries
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.callbacks import ModelCheckpoint
from keras.models import model_from_json
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Reshape
from keras.models import Model
from keras import backend as K
import numpy as np

Using TensorFlow backend.


In [2]:
### Loading Data
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = np.load(r'right_x_train.npy')
x_test = np.load(r'right_x_test.npy')

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

print('Training data shape: ', x_train.shape)
print('Testing data shape : ', x_test.shape)

Training data shape:  (60000, 28, 28)
Testing data shape :  (10000, 28, 28)


In [3]:
input_img = Input(shape=(28, 28, 1))  # adapt this if using `channels_first` image data format

x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# at this point the representation is (4, 4, 8) i.e. 128-dimensional

x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
### Training AE
from keras.callbacks import TensorBoard
x_train = x_train.reshape(60000,28,28,1)
x_test = x_test.reshape(10000,28,28,1)
autoencoder.fit(x_train, x_train,
                epochs=50,
                batch_size=128,
                shuffle=True,
                validation_data=(x_test, x_test),
                callbacks=[TensorBoard(log_dir='/tmp/autoencoder')])

Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/50
60000/60000 [==============================] - 19s 318us/step - loss: 0.1230 - val_loss: 0.0917
Epoch 2/50
60000/60000 [==============================] - 17s 286us/step - loss: 0.0843 - val_loss: 0.0754
Epoch 3/50
60000/60000 [==============================] - 17s 286us/step - loss: 0.0744 - val_loss: 0.0693
Epoch 4/50
60000/60000 [==============================] - 17s 287us/step - loss: 0.0692 - val_loss: 0.0660
Epoch 5/50
60000/60000 [==============================] - 17s 286us/step - loss: 0.0662 - val_loss: 0.0647
Epoch 6/50
60000/60000 [==============================] - 17s 288us/step - loss: 0.0639 - val_loss: 0.0628
Epoch 7/50
60000/60000 [==============================] - 17s 287us/step - loss: 0.0622 - val_loss: 0.0603
Epoch 8/50
60000/60000 [==============================] - 17s 286us/step - loss: 0.0608 - val_loss: 0.0623
Epoch 9/50
60000/60000 [======================

In [5]:
### Get Encoded features fron AE
encoder = Model(input_img, encoded)
encoded_imgs_test = encoder.predict(x_test)
encoded_imgs_train = encoder.predict(x_train)
print(encoded_imgs_test.shape)
print(encoded_imgs_train.shape)

(10000, 4, 4, 8)
(60000, 4, 4, 8)


In [6]:
encoded_imgs_test[0]

array([[[0.01862454, 0.28691977, 0.2043209 , 0.2935902 , 0.0253025 ,
         0.44627056, 0.05362864, 0.20643257],
        [0.1447528 , 0.9431563 , 0.9180924 , 0.6723197 , 0.30765715,
         1.160103  , 0.27315098, 0.4134518 ],
        [0.23307937, 1.3198574 , 0.9859771 , 0.91113317, 0.8025465 ,
         1.0090364 , 0.44489092, 0.3981384 ],
        [0.2055982 , 0.7569577 , 0.43345398, 0.8960943 , 0.39467153,
         0.08049189, 0.34654891, 0.        ]],

       [[0.0179552 , 0.2357165 , 0.2326038 , 0.3778486 , 0.08421934,
         0.3476381 , 0.33062536, 0.38333726],
        [1.0496483 , 0.91904205, 1.4776417 , 0.        , 0.56679153,
         1.9022161 , 1.9693677 , 0.4876135 ],
        [1.6330571 , 1.2561822 , 1.4573786 , 0.        , 1.8751793 ,
         2.1343753 , 1.9870944 , 0.70977825],
        [0.16864365, 0.3282979 , 0.60385305, 0.25274906, 0.76358175,
         0.34687868, 0.80991054, 1.5299995 ]],

       [[0.07421267, 0.22356161, 0.23391554, 0.16032383, 0.0802457 ,
       

In [7]:
encoded_imgs_test = encoded_imgs_test.reshape(10000,128)
encoded_imgs_train = encoded_imgs_train.reshape(60000,128)
print(encoded_imgs_test.shape)
print(encoded_imgs_train.shape)

(10000, 128)
(60000, 128)


In [8]:
np.save('right_test.npy',encoded_imgs_test)
np.save('right_train.npy',encoded_imgs_train)